# SQL

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import OpenAI, SelfAskWithSearchChain, LLMMathChain
from sqlalchemy.engine import Engine, create_engine
from langchain.agents import initialize_agent, Tool
import os

In [3]:
import sys
 # setting path
sys.path.append('..')
sys.path.append('../modules')
sys.path.append('../modules/database/')

from modules.database.sql_chain import SQLDatabaseChain
from modules.database.sql_database import SQLDatabase

In [9]:
# connect to remote postgres database
db = SQLDatabase.from_uri(os.getenv("DB_CONNECTION_STRING"))
print("Tables found:", db._all_tables)
db_chain = SQLDatabaseChain(llm=OpenAI(temperature=0), database=db, verbose=True, debug=False)

Tables found: ['users', 'templates', 'models', 'generations']


In [10]:
# Load the tool configs that are needed.
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about data in your database"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [11]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [13]:
agent.run("We had a bug yesterday. Are there any users that are stuck in training?")



> Entering new ZeroShotAgent chain...
We had a bug yesterday. Are there any users that are stuck in training?
Thought: I need to check the database to see if any users are stuck in training.
Action: Database
Action Input: Check for users stuck in training

> Entering new SQLDatabaseChain chain...
Check for users stuck in training 
SQLQuery: SELECT users.id, users.email, models.status FROM users INNER JOIN models ON users.id = models.user_id WHERE models.status = 'training';
SQLResult: [(UUID('205c5203-00f8-4cde-bed6-87f4000a6f05'), '205c5203-00f8-4cde-bed6-87f4000a6f05@gmail.com', 'training'), (UUID('db53a306-324a-432e-b95b-b3efbc8fdfd7'), 'db53a306-324a-432e-b95b-b3efbc8fdfd7@gmail.com', 'training'), (UUID('dc225eea-0b69-4b35-ad27-73b356fe9cd8'), 'dc225eea-0b69-4b35-ad27-73b356fe9cd8@gmail.com', 'training'), (UUID('d0782554-1fec-491a-8d0a-1ca01d50e7d8'), 'd0782554-1fec-491a-8d0a-1ca01d50e7d8@gmail.com', 'training'), (UUID('71d0195a-da10-44e5-aac4-b822a2965939'), '71d0195a-da10-44e5-

'Yes, there are 15 users stuck in training.'